In [1]:
import numpy as np
import cv2
import matplotlib.pyplot as plt
from skimage.segmentation import clear_border
from skimage import measure
from skimage.measure import label,regionprops
from scipy import ndimage as ndi
from scipy.ndimage import measurements, center_of_mass, binary_dilation, zoom
import plotly.graph_objects as go
from PIL import Image
import numpy as np
import matplotlib.pyplot as plt
from skimage import measure, morphology
import scipy.ndimage as ndimage
import cv2

In [2]:
def segmentBall(frame_num):
    image = cv2.imread(f'ryan_data/frame_{frame_num}.jpg', cv2.IMREAD_GRAYSCALE) # frame_350.jpg

    # Preprocess the image
    # Apply any necessary filtering, denoising, or smoothing operations
    # For example, using a Gaussian filter:
    image_filtered = ndimage.gaussian_filter(image, sigma=1)

    # Apply thresholding to create a binary mask
    mask = image_filtered > 0.1  # Replace threshold_value with an appropriate value

    # Remove small components from the mask
    mask = morphology.remove_small_objects(mask, min_size=50, connectivity=0)

    # Label the connected components
    mask_labeled = morphology.label(mask, connectivity=1)

    # Get the bounding boxes of each component
    slices = ndi.find_objects(mask_labeled)

    # Create a new mask without the components touching the border
    new_mask = np.zeros_like(mask_labeled)
    for i, slc in enumerate(slices, start=1):
        is_border = False
        for s, dim in zip(slc, mask_labeled.shape):
            if s.start == 0 or s.stop == dim:
                is_border = True
                break
        if not is_border:
            new_mask[slc] = mask_labeled[slc] * (mask_labeled[slc] == i)

    # Replace the original mask with the new mask
    mask = new_mask > 0

    slc = mask_labeled
    rps = regionprops(slc)
    areas = [r.area for r in rps]
    np.argsort(areas)[::-1]

    slc = mask_labeled
    rps = regionprops(slc)
    areas = [r.area for r in rps]
    idxs = np.argsort(areas)[::-1] # we want largest to smallest

    new_slc = np.zeros_like(slc)
    for i in idxs[:5]:
        new_slc[tuple(rps[i].coords.T)] = i+1

    mask = mask_labeled > 0
    mask = np.vectorize(ndi.binary_fill_holes, signature='(n,m)->(n,m)')(mask)

    # Find components with area between 0.1% and 1% of the total image area
    labels = label(mask, connectivity=1, background=0)
    rps = regionprops(labels)
    areas = np.array([r.area for r in rps])
    small_components = np.where((areas / mask.size < 0.001) & (areas / mask.size > 0.0003))[0]

    # Create a new mask with only the small components
    small_components_mask = np.zeros_like(labels, dtype=bool)
    for idx in small_components:
        small_components_mask[labels == idx + 1] = True

    small_eccentricities = np.array([rps[i].eccentricity for i in small_components])
    small_circularity = 1 - small_eccentricities

    # Find the index of the most circular component within small_components
    most_circular_idx = np.argmax(small_circularity)

    # Get the label of the most circular component
    most_circular_component = small_components[most_circular_idx]

    # Create a mask for the most circular component
    small_components_mask = labels == most_circular_component + 1

    img = cv2.imread(f'ryan_data/org_frame_{frame_num}.jpg')
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    img[~small_components_mask] = 0

    cv2.imwrite(f'frames/f_{frame_num}.jpg', img)


In [4]:
# for i in range(0, 350):
#     segmentBall(i)

segmentBall(350)

TypeError: '>' not supported between instances of 'NoneType' and 'float'